In [1]:
import pandas as pd
import numpy as np 
import re
import random as rd
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta

# Import Dataset
importiamo i csv all'interno di dataframe utilizzando la funzione della libreria Pandas

In [2]:
loans_lenders = pd.read_csv("loans_lenders.csv")

In [3]:
loans = pd.read_csv('loans.csv')

In [4]:
lenders = pd.read_csv('lenders.csv')

In [5]:
country = pd.read_csv("country_stats.csv")

# Punto 1
Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender

Per il dataset "loans_lenders" si osserva come il dataset sia formato da due colonne. Una contenente gli ID dei lenders, e una contenete il loro user_name. Ogni record della seconda colonna è una serie di nomi separati dalla virgola.

In [4]:
loans_lenders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387432 entries, 0 to 1387431
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   loan_id  1387432 non-null  int64 
 1   lenders  1387432 non-null  object
dtypes: int64(1), object(1)
memory usage: 21.2+ MB


In [5]:
{column: {type(val) for val in loans_lenders[column]} for column in loans_lenders}

{'loan_id': {int}, 'lenders': {str}}

In [6]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Controlliamo la presenza di valori nulli all'interno del dataset

In [7]:
loans_lenders.isnull().values.any()

False

Nel contesto di dominio la colonna "lenders" individua i nickname dei lenders. Con una prima osservazione possiamo pensare che sia un valore stringa alfanumerico e quindi un solo valore per utente compost in parte da lettere e in parte da numeri senza nessuno spazio tra di loro. Poichè la dimensione del dataset non ci permette di dare uno sguardo veloce a tutti, ci curiamo di tutto ciò tramite la seguente riga di codice. 
Con la seguente list comprehension andiamo a prendere ogni campo della colonna 'lenders' e valutiamo se il numero di parole comprese tra due virgole sono al più una sola. 
Per farlo utilizziamo una semplice regex dopo aver splittato i valori in base alla virgola e calcoliamo quanti di essi sono diversi da 1. 
In questo caso vediamo che nessuno elemento supera il singolo elemento, quindi confermiamo ciò che abbiamo pensato

In [11]:
[check for check in [[len([word]) for word in re.findall(r'\w+', i) if len([word]) != 1 for i in i.split(',')] for i in loans_lenders['lenders']] if check!= []] == []

True

Iniziamo a crearci una lista con all'interno ogni singolo user_name separato dalla virgola. Questo ci aiuterà successivamente ad effettuare la funzione "pd.explode" sulla colonna che ci consentirà di dividere gli elementi della lista e creare un numero di righe tante quanti sono gli elementi della stessa lista, lasciando immutati i valori degli ID appartenti.

In [8]:
loans_lenders['lenders'] = loans_lenders.lenders.str.split(',').tolist()

In [8]:
loans_lenders['lenders'][0]

['muc888',
 ' sam4326',
 ' camaran3922',
 ' lachheb1865',
 ' rebecca3499',
 ' karlheinz4543',
 ' jerrydb',
 ' paula8951',
 ' gmct',
 ' amra9383',
 ' r3922',
 ' brian9451',
 ' shree8053',
 ' alan5513',
 ' oisin3389',
 ' helle8622',
 ' bo3186',
 ' ric8947',
 ' daniel98469874',
 ' nick9464',
 ' deborah12671549',
 ' matthew9831',
 ' john6330',
 ' john9479',
 ' mattiaslaven',
 ' jonathan2867',
 ' jason3883',
 ' highgrovechurch',
 ' maria3124',
 ' dino5102',
 ' jonathan7946',
 ' ann8187',
 ' bryan2669',
 ' john88459657',
 ' eddyphil',
 ' don9212',
 ' carolineandcolin9686',
 ' bent8782',
 ' raph8817',
 ' danielle2350']

Eseguendo un controllo sul risultato notiamo la presenza di spazi presenti prima della virgola. Questo potrebbe far incorrere in errore i successivi algoritmi, quindi tramite list comprehension andiamo ad eliminare il problema.

In [9]:
loans_lenders['lenders'] = [[x.strip(' ') for x in row] for row in loans_lenders['lenders']]

Nuovo check sempre sullo stesso elemento di prima per poter confrontare i risultati e valutare la correttezza della soluzione

In [12]:
loans_lenders['lenders'][0]

['muc888',
 'sam4326',
 'camaran3922',
 'lachheb1865',
 'rebecca3499',
 'karlheinz4543',
 'jerrydb',
 'paula8951',
 'gmct',
 'amra9383',
 'r3922',
 'brian9451',
 'shree8053',
 'alan5513',
 'oisin3389',
 'helle8622',
 'bo3186',
 'ric8947',
 'daniel98469874',
 'nick9464',
 'deborah12671549',
 'matthew9831',
 'john6330',
 'john9479',
 'mattiaslaven',
 'jonathan2867',
 'jason3883',
 'highgrovechurch',
 'maria3124',
 'dino5102',
 'jonathan7946',
 'ann8187',
 'bryan2669',
 'john88459657',
 'eddyphil',
 'don9212',
 'carolineandcolin9686',
 'bent8782',
 'raph8817',
 'danielle2350']

In [10]:
loans_lenders = loans_lenders.explode('lenders').reset_index()

In [20]:
loans_lenders.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


# Punto 2
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [13]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419607 entries, 0 to 1419606
Data columns (total 31 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   loan_id                          1419607 non-null  int64  
 1   loan_name                        1372049 non-null  object 
 2   original_language                1376356 non-null  object 
 3   description                      1376322 non-null  object 
 4   description_translated           966929 non-null   object 
 5   funded_amount                    1419607 non-null  float64
 6   loan_amount                      1419607 non-null  float64
 7   status                           1419607 non-null  object 
 8   activity_name                    1419607 non-null  object 
 9   sector_name                      1419607 non-null  object 
 10  loan_use                         1376334 non-null  object 
 11  country_code                     1419598 non-null 

In [14]:
{column: {type(val) for val in loans[column]} for column in loans}

{'loan_id': {int},
 'loan_name': {float, str},
 'original_language': {float, str},
 'description': {float, str},
 'description_translated': {float, str},
 'funded_amount': {float},
 'loan_amount': {float},
 'status': {str},
 'activity_name': {str},
 'sector_name': {str},
 'loan_use': {float, str},
 'country_code': {float, str},
 'country_name': {str},
 'town_name': {float, str},
 'currency_policy': {str},
 'currency_exchange_coverage_rate': {float},
 'currency': {str},
 'partner_id': {float},
 'posted_time': {str},
 'planned_expiration_time': {float, str},
 'disburse_time': {float, str},
 'raised_time': {float, str},
 'lender_term': {float},
 'num_lenders_total': {int},
 'num_journal_entries': {int},
 'num_bulk_entries': {int},
 'tags': {float, str},
 'borrower_genders': {float, str},
 'borrower_pictured': {float, str},
 'repayment_interval': {str},
 'distribution_model': {str}}

In [5]:
loans.isnull().values.any()

True

Notiamo che alcuni campi contengono due type diversi. Per questo andiamo a visualizzare quei campi considerando solo le righe con il il type diverso. Si vede come il diverso type è dovuto dall'esistenza di valori nulli

In [6]:
loans[loans['loan_name'].apply(lambda x: isinstance(x, float))].head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
56,417029,NaN,NaN,NaN,NaN,1225.0,1225.0,funded,Clothing,Clothing,...,2012-05-15 19:55:22.000 +0000,14.0,21,4,3,NaN,NaN,NaN,monthly,field_partner
106,962596,NaN,NaN,NaN,NaN,650.0,650.0,funded,Restaurant,Food,...,2015-10-17 23:48:20.000 +0000,27.0,18,4,2,"#Biz Durable Asset, #Technology, #Unique",NaN,NaN,monthly,field_partner
275,963073,NaN,Spanish,EL ES UN JOVEN PADRE DE FAMILIA QUE ACTUALMENT...,He is a young father who currently supports hi...,1175.0,1175.0,funded,Personal Medical Expenses,Health,...,2015-11-21 22:04:02.000 +0000,26.0,34,1,1,"#Health and Sanitation, #Parent, user_favorite...",male,true,monthly,field_partner
296,1154475,NaN,NaN,NaN,NaN,200.0,200.0,funded,Home Appliances,Personal Use,...,2016-09-22 03:51:06.000 +0000,8.0,8,2,1,"#Health and Sanitation, #Eco-friendly, #Techno...",NaN,NaN,monthly,field_partner
322,659763,NaN,NaN,NaN,NaN,375.0,375.0,funded,Farming,Agriculture,...,2014-01-23 22:46:20.000 +0000,10.0,10,1,1,"#Post-disbursed, #Parent",NaN,NaN,bullet,field_partner


In [7]:
loans[loans['original_language'].apply(lambda x: isinstance(x, float))].head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
56,417029,NaN,NaN,NaN,NaN,1225.0,1225.0,funded,Clothing,Clothing,...,2012-05-15 19:55:22.000 +0000,14.0,21,4,3,NaN,NaN,NaN,monthly,field_partner
106,962596,NaN,NaN,NaN,NaN,650.0,650.0,funded,Restaurant,Food,...,2015-10-17 23:48:20.000 +0000,27.0,18,4,2,"#Biz Durable Asset, #Technology, #Unique",NaN,NaN,monthly,field_partner
296,1154475,NaN,NaN,NaN,NaN,200.0,200.0,funded,Home Appliances,Personal Use,...,2016-09-22 03:51:06.000 +0000,8.0,8,2,1,"#Health and Sanitation, #Eco-friendly, #Techno...",NaN,NaN,monthly,field_partner
322,659763,NaN,NaN,NaN,NaN,375.0,375.0,funded,Farming,Agriculture,...,2014-01-23 22:46:20.000 +0000,10.0,10,1,1,"#Post-disbursed, #Parent",NaN,NaN,bullet,field_partner
355,423391,NaN,NaN,NaN,NaN,1200.0,1200.0,funded,Carpentry,Construction,...,2012-05-26 18:35:27.000 +0000,17.0,38,3,2,NaN,NaN,NaN,monthly,field_partner


In [9]:
loans[loans['country_code'].apply(lambda x: isinstance(x, float))].head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
82889,991853,Elizabeth,English,Elizabeth is excited to have the opportunity t...,Elizabeth is excited to have the opportunity t...,3325.0,3325.0,funded,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,EEnhana,shared,0.1,NAD,437.0,2015-12-08 19:08:18.000 +0000,2016-01-21 02:20:06.000 +0000,2016-01-05 08:00:00.000 +0000,2015-12-22 03:53:32.000 +0000,7.0,120,2,1,user_favorite,female,true,bullet,field_partner
156970,513472,Gottlieb,English,"Gottlieb works for ELEPHANT ENERGY, a Barefoot...",NaN,7150.0,7150.0,funded,Renewable Energy Products,Retail,to buy and sell Barefoot Power's solar lightin...,NaN,Namibia,Windhoek,shared,0.1,USD,212.0,2013-01-02 23:25:03.000 +0000,2013-02-02 00:10:01.000 +0000,2013-03-08 08:00:00.000 +0000,2013-01-03 15:10:46.000 +0000,16.0,234,2,1,NaN,male,true,irregular,field_partner
598087,851360,Elizabeth,English,Elizabeth has five years of experience as an o...,Elizabeth has five years of experience as an o...,4150.0,4150.0,funded,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,EEnhana,shared,0.1,NAD,437.0,2015-03-05 20:23:50.000 +0000,2015-04-24 14:00:04.000 +0000,2015-06-03 07:00:00.000 +0000,2015-03-25 15:10:03.000 +0000,6.0,162,2,1,"user_favorite, user_favorite",female,true,bullet,field_partner
684876,1068159,Theo,English,"With his second loan, Theo was able to open a ...","With his second loan, Theo was able to open a ...",4000.0,4000.0,funded,Wholesale,Wholesale,purchase solar lighting products for sale to l...,NaN,Namibia,Rundu,shared,0.1,NAD,437.0,2016-05-12 21:19:41.000 +0000,2016-06-14 14:10:06.000 +0000,2016-06-10 07:00:00.000 +0000,2016-05-15 17:36:49.000 +0000,7.0,150,1,1,"#Eco-friendly, #Repeat Borrower, #Technology, ...",male,true,bullet,field_partner
971827,998555,Theo,English,Theo is so thankful for his first loan and for...,Theo is so thankful for his first loan and for...,3325.0,3325.0,funded,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,Rundu,shared,0.1,NAD,437.0,2015-12-21 19:59:37.000 +0000,2016-01-21 02:20:17.000 +0000,2016-01-05 08:00:00.000 +0000,2015-12-22 05:58:09.000 +0000,7.0,126,2,1,NaN,male,true,bullet,field_partner


In [11]:
loans['disburse_time']  = pd.to_datetime(loans['disburse_time']).dt.date
loans['planned_expiration_time']  = pd.to_datetime(loans['planned_expiration_time']).dt.date

In [12]:
loans.insert(loans.columns.get_loc('disburse_time') + 1, 'duration', (loans['planned_expiration_time'] - loans['disburse_time']).dt.days)

# Punto 3
Find the lenders that have funded at least twice.

In [17]:
df_id_founded = loans_lenders.merge(loans[loans['status'] == 'funded']['loan_id'], how='left', on='loan_id').groupby('lenders').count().reset_index()

In [18]:
df_id_founded[df_id_founded['loan_id'] >= 2]

,lenders,index,loan_id
0,000,40,40
1,00000,39,39
2,0002,70,70
4,0101craign0101,71,71
5,0132575,4,4
...,...,...,...
1383792,zzinnia,28,28
1383793,zzivnk7094,4,4
1383794,zzmcfate,1730,1730
1383796,zzrvmf8538,15,15


# Punto 4
For each country, compute how many loans have involved that country as borrowers

In [19]:
loans.groupby('country_name')['loan_id'].count().reset_index()

,country_name,loan_id
0,Afghanistan,2337
1,Albania,3075
2,Armenia,13952
3,Azerbaijan,10172
4,Belize,218
...,...,...
91,Vietnam,21839
92,Virgin Islands,2
93,Yemen,4206
94,Zambia,1277


# Punto 5
For each country, compute the overall amount of money borrowed

In [22]:
loans.groupby(['country_name'])['loan_amount'].sum().reset_index()

,country_name,loan_amount
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0
...,...,...
91,Vietnam,24681100.0
92,Virgin Islands,10000.0
93,Yemen,3444000.0
94,Zambia,1978975.0


# Punto 6 
Like the previous point, but expressed as a percentage of the overall amount lent

In [23]:
df_sum['perc'] = round((df_sum['loan_amount']/df_sum['loan_amount'].sum())*100, 4)

In [24]:
df_sum

,country_name,loan_amount,perc
0,Afghanistan,1967950.0,0.1666
1,Albania,4307350.0,0.3646
2,Armenia,22950475.0,1.9426
3,Azerbaijan,14784625.0,1.2514
4,Belize,150175.0,0.0127
...,...,...,...
91,Vietnam,24681100.0,2.0891
92,Virgin Islands,10000.0,0.0008
93,Yemen,3444000.0,0.2915
94,Zambia,1978975.0,0.1675


# Punto 7
Like the three previous points, but split for each year (with respect to disburse time)

# 7.1
For each country, compute how many loans have involved that country as borrowers, but split for each year (with respect to disburse time)

In [25]:
loans['disburse_year'] = pd.DatetimeIndex(loans['disburse_time']).year

In [26]:
loans.groupby(['country_name', 'disburse_year'])['loan_id'].count().reset_index(name="count")

,country_name,disburse_year,count
0,Afghanistan,2007.0,408
1,Afghanistan,2008.0,370
2,Afghanistan,2009.0,678
3,Afghanistan,2010.0,632
4,Afghanistan,2011.0,247
...,...,...,...
743,Zimbabwe,2013.0,426
744,Zimbabwe,2014.0,2078
745,Zimbabwe,2015.0,600
746,Zimbabwe,2016.0,808


# 7.2
For each country, compute the overall amount of money borrowed, but split for each year (with respect to disburse time)

In [27]:
df_sum_year = loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum().reset_index(name="loan_amount_per_year")

In [28]:
df_sum_year

,country_name,disburse_year,loan_amount_per_year
0,Afghanistan,2007.0,194975.0
1,Afghanistan,2008.0,365375.0
2,Afghanistan,2009.0,585125.0
3,Afghanistan,2010.0,563350.0
4,Afghanistan,2011.0,245125.0
...,...,...,...
743,Zimbabwe,2013.0,678525.0
744,Zimbabwe,2014.0,1311575.0
745,Zimbabwe,2015.0,723625.0
746,Zimbabwe,2016.0,788600.0


# 7.3
Like the previous point, but expressed as a percentage of the overall amount lent and split for each year (with respect to disburse time)

In [29]:
df_sum_year['perc'] = round((df_sum_year['loan_amount_per_year']/df_sum_year['loan_amount_per_year'].sum())*100, 4)

In [30]:
df_sum_year

,country_name,disburse_year,loan_amount_per_year,perc
0,Afghanistan,2007.0,194975.0,0.0167
1,Afghanistan,2008.0,365375.0,0.0312
2,Afghanistan,2009.0,585125.0,0.0500
3,Afghanistan,2010.0,563350.0,0.0481
4,Afghanistan,2011.0,245125.0,0.0209
...,...,...,...,...
743,Zimbabwe,2013.0,678525.0,0.0580
744,Zimbabwe,2014.0,1311575.0,0.1121
745,Zimbabwe,2015.0,723625.0,0.0618
746,Zimbabwe,2016.0,788600.0,0.0674


# Punto 8
For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount

In [13]:
loans['amount_lender'] = loans.eval('loan_amount/num_lenders_total')

In [14]:
totale_lenders_amount = loans[['loan_id','amount_lender']].merge(loans_lenders,'left',on='loan_id').groupby(['lenders'])['amount_lender'].sum().round(2).reset_index(name="lender_amount")

In [16]:
totale_lenders_amount

,lenders,lender_amount
0,000,1563.88
1,00000,1249.95
2,0002,2201.18
3,00mike00,38.46
4,0101craign0101,2424.09
...,...,...
1383794,zzmcfate,60150.83
1383795,zzpaghetti9994,44.25
1383796,zzrvmf8538,551.83
1383797,zzzsai,236.89


# Punto 9
For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [26]:
lenders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349174 entries, 0 to 2349173
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   permanent_name     object 
 1   display_name       object 
 2   city               object 
 3   state              object 
 4   country_code       object 
 5   member_since       int64  
 6   occupation         object 
 7   loan_because       object 
 8   loan_purchase_num  float64
 9   invited_by         object 
 10  num_invited        int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 197.2+ MB


In [27]:
{column: {type(val) for val in lenders[column]} for column in lenders}

{'permanent_name': {str},
 'display_name': {float, str},
 'city': {float, str},
 'state': {float, str},
 'country_code': {float, str},
 'member_since': {int},
 'occupation': {float, str},
 'loan_because': {float, str},
 'loan_purchase_num': {float},
 'invited_by': {float, str},
 'num_invited': {int}}

In [28]:
loans.isnull().values.any()

True

In [15]:
loan_distribution_per_country_code = loans[['country_code', 'loan_amount']]
loan_distribution_per_country_code

,country_code,loan_amount
0,PH,125.0
1,HN,400.0
2,PK,400.0
3,KG,625.0
4,PH,425.0
...,...,...
1419602,KE,400.0
1419603,PH,300.0
1419604,GH,2425.0
1419605,PH,100.0


In [16]:
distr = loan_distribution_per_country_code.country_code.value_counts(normalize=True)
missing = lenders['country_code'].isnull()
lenders.loc[missing,'country_code'] = np.random.choice(distr.index, size=len(lenders[missing]),p=distr.values)

In [17]:
country_overall = totale_lenders_amount.join(lenders[['permanent_name','country_code']].set_index('permanent_name'), on='lenders').groupby(['country_code'])['lender_amount'].sum().reset_index().merge(loans.groupby(['country_code'])['loan_amount'].sum().reset_index(name="borrower_amount"), how = 'left', on = 'country_code').fillna(0).round(2)

In [18]:
country_overall['difference'] = country_overall.eval('lender_amount - borrower_amount')

In [27]:
country_overall

,country_code,lender_amount,borrower_amount,difference
0,AD,4478.19,0.0,4478.19
1,AE,1582174.85,0.0,1582174.85
2,AF,1002059.72,1967950.0,-965890.28
3,AG,550.84,0.0,550.84
4,AI,322.58,0.0,322.58
...,...,...,...,...
222,YE,555940.19,3444000.0,-2888059.81
223,YT,55.22,0.0,55.22
224,ZA,1191759.04,1006525.0,185234.04
225,ZM,177435.95,1978975.0,-1801539.05


# Punto 10
Which country has the highest ratio between the difference computed at the previous point and the population?

In [19]:
country_analysis = country.merge(country_overall[['country_code','difference']], how = 'left', on = 'country_code').fillna(0)

In [20]:
country_analysis['ratio'] = country_analysis.eval('difference/population').round(2)

In [21]:
country_analysis.iloc[country_analysis['ratio'].idxmax]

country_name                              Norway
country_code                                  NO
country_code3                                NOR
continent                                 Europe
region                           Northern Europe
population                               5305383
population_below_poverty_line                  0
hdi                                     0.949423
life_expectancy                           81.711
expected_years_of_schooling              17.6719
mean_years_of_schooling                  12.7464
gni                                      67614.4
kiva_country_name                         Norway
difference                           1.72257e+07
ratio                                       3.25
Name: 162, dtype: object

# Punto 11
Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [22]:
country_analysis['ratio_poverty_line']= country_analysis.eval('difference/ ((100 - population_below_poverty_line)/100) * population')

In [23]:
country_analysis[country_analysis['ratio_poverty_line']==country_analysis['ratio_poverty_line'].max()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,difference,ratio,ratio_poverty_line
6,United States,US,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.53815,13.21789,53245.07696,United States,4.554612e+08,1.4,1.740621e+17


# Punto 12
For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD 31 / (31+365+30) = 363.85 for 2016, 5000USD 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018

In [78]:
loan_analysis = loans[loans['duration'] >= 0][['loan_id','disburse_time','planned_expiration_time','duration','loan_amount']].dropna()

In [80]:
loan_analysis['planned_expiration_time_year'] = pd.to_datetime(loan_analysis['planned_expiration_time']).dt.year.astype('Int64')
loan_analysis['disburse_time_year'] = pd.to_datetime(loan_analysis['disburse_time']).dt.year.astype('Int64')

In [101]:
loan_analysis['delta_year'] = loan_analysis['planned_expiration_time_year'] - loan_analysis['disburse_time_year']

In [132]:
loan_analysis

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,planned_expiration_time_year,disburse_time_year,delta_year
0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1
1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1
2,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014,0
3,659347,2014-01-17,2014-02-21,35.0,625.0,2014,2014,0
4,656933,2013-12-17,2014-02-13,58.0,425.0,2014,2013,1
...,...,...,...,...,...,...,...,...
1419602,988180,2015-11-23,2016-01-02,40.0,400.0,2016,2015,1
1419603,988213,2015-11-24,2016-01-02,39.0,300.0,2016,2015,1
1419604,989109,2015-11-13,2016-01-03,51.0,2425.0,2016,2015,1
1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1


In [137]:
loan_analysis['year_range'] = loan_analysis.apply(lambda x: list(range(x['disburse_time_year'], x['planned_expiration_time_year'] +1)), axis = 1)

In [139]:
loan_analysis = loan_analysis.explode('year_range').reset_index()

In [140]:
loan_analysis

,index,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,planned_expiration_time_year,disburse_time_year,delta_year,year_range
0,0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1,2013
1,0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1,2014
2,1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1,2013
3,1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1,2014
4,2,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014,0,2014
...,...,...,...,...,...,...,...,...,...,...
1181799,1419604,989109,2015-11-13,2016-01-03,51.0,2425.0,2016,2015,1,2016
1181800,1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1,2015
1181801,1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1,2016
1181802,1419606,989240,2015-11-03,2016-01-03,61.0,175.0,2016,2015,1,2015


In [144]:
loan_analysis['leap_year'] = loan_analysis['year_range'].apply(lambda date: calendar.isleap(date))

In [173]:
loan_analysis

,index,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,planned_expiration_time_year,disburse_time_year,delta_year,year_range,leap_year
0,0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1,2013,False
1,0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1,2014,False
2,1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1,2013,False
3,1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1,2014,False
4,2,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014,0,2014,False
...,...,...,...,...,...,...,...,...,...,...,...
1181799,1419604,989109,2015-11-13,2016-01-03,51.0,2425.0,2016,2015,1,2016,True
1181800,1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1,2015,False
1181801,1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1,2016,True
1181802,1419606,989240,2015-11-03,2016-01-03,61.0,175.0,2016,2015,1,2015,False


In [180]:
def calcolo_loan_partizionato(data_inizio, data_fine, duration, loan_amount, anno_inizio, anno_fine, delta_anno, anno_range, leap): 
    
    somma_loan = 0 
    data_fine_anno_disburse = dt.datetime.date(dt.datetime(anno_inizio + 1, 1, 1))
    data_inizio_anno_expiration = dt.datetime.date(dt.datetime(anno_fine -1 , 12, 31))
    
    if  (anno_range ==  anno_inizio): 
        somma_loan = loan_amount *(((data_fine_anno_disburse - data_inizio).days) / duration)
    
    elif (anno_range ==  anno_fine): 
        somma_loan = loan_amount *(((data_fine - data_inizio_anno_expiration).days) / duration)
        
    elif (anno_range != anno_inizio and anno_range != anno_fine and leap == False):
        somma_loan = (loan_amount *365) / duration
    
    else: 
        somma_loan = (loan_amount *366) / duration
    
    return round(somma_loan, 2)

In [181]:
loan_analysis['loan partizionato'] = loan_analysis.apply(lambda x: x['loan_amount'] if x['delta_year'] == 0 else calcolo_loan_partizionato(x['disburse_time'], x['planned_expiration_time'], x['duration'], x['loan_amount'],  x['disburse_time_year'], x['planned_expiration_time_year'], x['delta_year'], x['year_range'], x['leap_year']), axis=1)

In [182]:
loan_analysis

,index,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,planned_expiration_time_year,disburse_time_year,delta_year,year_range,leap_year,loan partizionato
0,0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1,2013,False,23.15
1,0,657307,2013-12-22,2014-02-14,54.0,125.0,2014,2013,1,2014,False,104.17
2,1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1,2013,False,50.00
3,1,657259,2013-12-20,2014-03-26,96.0,400.0,2014,2013,1,2014,False,354.17
4,2,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014,0,2014,False,400.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1181799,1419604,989109,2015-11-13,2016-01-03,51.0,2425.0,2016,2015,1,2016,True,142.65
1181800,1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1,2015,False,93.65
1181801,1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2016,2015,1,2016,True,7.94
1181802,1419606,989240,2015-11-03,2016-01-03,61.0,175.0,2016,2015,1,2015,False,169.26


In [187]:
loan_analysis.groupby(['year_range'])['loan partizionato'].sum().reset_index()